# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [2]:
keywords = ['python', 'парсинг']

list_date, list_header, list_url = [], [], []
res_dict = {'data': list_date,
         'header': list_header,
         'url': list_url}

url = 'https://habr.com/ru/all/'
web_page = requests.get(url)

if web_page.status_code != 200:
    print(f'Страница {url} недоступна.')
    
else:
    soup = BeautifulSoup(web_page.text, 'html.parser') # Создание объекта стартовой страницы
    start_page = soup.find('div', class_='content_left js-content_left') # Скрапинг стартовой страницы
    
    article_url = start_page.find_all('a', class_='post__title_link') # Получаем все ссылки на новости со стартовой страницы
    list_article_url = list() # Список ссылок
    for url in article_url:
        list_article_url.append(url.get('href'))
    
    for article in list_article_url: # Проходимся по каждой статье
        article_page = requests.get(article)
        soup = BeautifulSoup(article_page.text, 'html.parser') # Создание объекта страницы с новостью
        
        for keyword in keywords: # Проверяем наличие ключевых слов в тексте статьи
            article_text = soup.find('div', class_='post__text post__text-html post__text_v1').get_text()
            
            if keyword in article_text.lower(): # Если слово есть, то добавляем в списки
                list_date.append(soup.find('span', class_='post__time').get_text()) # получаем дату
                list_header.append(soup.find('span', class_='post__title-text').get_text()) # получаем дату
                list_url.append(article)
    
    res_dict['data'] = list_date
    res_dict['header'] = list_header
    res_dict['url'] = list_url


res_df = pd.DataFrame(res_dict)
res_df

,data,header,url
0,сегодня в 19:50,FOSS News №28 – дайджест новостей свободного и...,https://habr.com/ru/post/514434/
1,сегодня в 15:43,"Как не потерять ход времени, работая за компью...",https://habr.com/ru/post/514414/
2,сегодня в 08:21,Мониторинг демон на Asyncio + Dependency Injec...,https://habr.com/ru/post/514384/
3,сегодня в 08:21,Мониторинг демон на Asyncio + Dependency Injec...,https://habr.com/ru/post/514384/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [3]:
import json
import datetime as dt

emails = ['ivan@mail.ru', 'jhon@google.com']

def checking_email (email):
    url = 'https://digibody.avast.com/v1/web/leaks'
    data = {'email': email}
    res_page = requests.post(url, data=json.dumps(data))
    return res_page.json()


list_email, list_date, list_source, list_description = [], [], [], []
res_check_dict = {'email': list_email,
                  'date': list_date,
                  'source': list_source,
                  'description': list_description}

for email in emails:
    result = checking_email(email)
    
    for element in result['value']:
        list_email.append(element['username'])
        list_date.append(dt.datetime.fromtimestamp(element['leak_info']['date']/1000))
        list_source.append(element['leak_info']['title'])
        list_description.append(element['leak_info']['description'])

res_check_dict['email'] = list_email
res_check_dict['date'] = list_date
res_check_dict['source'] = list_source
res_check_dict['description'] = list_description
    
res_check_df = pd.DataFrame(res_check_dict)
res_check_df

,email,date,source,description
0,ivan__12,2020-07-23 03:00:00,LiveAuctioneers,"In June 2020, the online marketplace LiveAucti..."
1,superivan123,2019-05-23 03:00:00,LiveJournal,"In 2017, social network LiveJournal's database..."
2,luka68,2019-05-23 03:00:00,LiveJournal,"In 2017, social network LiveJournal's database..."
3,ghyid,2019-05-23 03:00:00,LiveJournal,"In 2017, social network LiveJournal's database..."
4,thuyr,2019-05-23 03:00:00,LiveJournal,"In 2017, social network LiveJournal's database..."
...,...,...,...,...
74,jhon@google.com,2017-06-01 03:00:00,Sensitive Source,This source has been marked as sensitive due t...
75,jhon@google.com,2019-02-06 03:00:00,AP MYR and Zabugor Combo List,"On January 7, 2019, an online user named Sanix..."
76,jhon@google.com,2020-01-09 03:00:00,Sensitive Source,This source has been marked as sensitive due t...
77,earl112,2016-10-20 03:00:00,Neopets,"In May 2016, it was reported that approximatel..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
